<a href="https://colab.research.google.com/github/anish2cod/NerualNetwork/blob/main/neuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
np.poly1d([1,0,1])


10

In [18]:
np.random.rand(10,2)

array([[0.54662757, 0.87374003],
       [0.73187284, 0.73639702],
       [0.86986912, 0.17507052],
       [0.97188276, 0.98514165],
       [0.51193933, 0.31710679],
       [0.97229989, 0.42176488],
       [0.48036652, 0.36590021],
       [0.01376671, 0.34862218],
       [0.27605245, 0.1197345 ],
       [0.98999526, 0.27122846]])

In [12]:
class Neuron:
  def __init__(self, value = 0):
    self.value = value


In [23]:
class Layer:
  def __init__(self, number_of_neurons,input_size , activation,dropout,input=None):
    if input == None:
      self.neurons = [Neuron() for _ in range(number_of_neurons)]
    else:
      self.neurons = [Neuron(val) for val in input]
    self.weights = np.random.rand(number_of_neurons , input_size)
    self.activation = activation
    self.dropout = dropout


In [25]:
l1 = Layer(10,5,'relu',.3,[i**2 for i in range(10)])
l1.neurons

In [27]:
l1.neurons[2].value

4